In [ ]:
%%writefile Baswana_Sen_Serial.cpp
// G --> number of nodes, edges, weight of each edge
#include <vector>
#include <utility>
#include <set>
#include <climits>
#include <stdio.h>
#include <time.h>
#include <string.h>
#include <math.h>
#include <stdlib.h>
#include <chrono>
#include <bits/stdc++.h>
using namespace std;
#define ll long long int
#define LLI_MAX 0x7fffffffffffffff
#define K 3
#define N 25
#define ERDOS_RENYI_PROBA 0.5
#define ERDOS_RENYI_MAX_WEIGHT 50
#define PRINT_STEPS true
vector<pair<ll, ll>> G[N + 10];
vector<pair<ll, ll>> H[N + 10];
vector<pair<ll, ll>> residual_graph[N + 10];
vector<ll> clustering;
// clustering[0] : 0, 1
// clustering[1] : 3 , 4
vector<ll> weight[N + 20][N + 20];
pair<map<ll, ll>, map<ll, ll>> lightest_edge_dicts();
void generate_graph()
{

    for (int a = 1; a <= N; a++)
    {
        for (int b = a + 1; b <= N; b++)
        {
            double ranv = ((double)rand()) / RAND_MAX;
            if (ranv <= ERDOS_RENYI_PROBA)
            {
                long long int wgt = 1 + (rand() % ERDOS_RENYI_MAX_WEIGHT);
                G[a].push_back({b, wgt});
                G[b].push_back({a, wgt});
                weight[a][b].push_back(wgt);
                weight[b][a].push_back(wgt);
            }
        }
    }
}

void setup_residual_graph()
{
    for (int i = 1; i <= N; i++)
    {
        for (auto u : G[i])
        {
            residual_graph[i].push_back({u.first, u.second});
        }
    }
}
pair<map<ll, ll>, map<ll, ll>> lightest_edge_dicts(int v)
{
    map<ll, ll> lightest_edge_neighbour, lightest_edge_weight;

    for (int i = 1; i <= N; i++)
    {
        ll neighbor_center = clustering[i];
        ll weight = residual_graph[v][i].second;

        if (
            (!lightest_edge_weight[neighbor_center]) || (weight < lightest_edge_weight[neighbor_center]))
        {
            lightest_edge_neighbour[neighbor_center] = i;
            lightest_edge_weight[neighbor_center] = weight;
        }
    }

    return {lightest_edge_neighbour, lightest_edge_weight};
}

void add_edge_to_spanner(ll u, ll v, ll weight)
{
    H[u].push_back({v, weight});
    H[v].push_back({u, weight});
}

void spanner(double stretch, ll weight = 0, ll seed = 0)
{
    if (stretch < 1)
    {
        cout << "value of stretch cannot be less than 1\n";
        return;
    }
    int k = (stretch + 1) / 2;
    //cout << "Inside Spanner..............................";
    setup_residual_graph();
    for (int i = 1; i <= N; i++)
    {
        clustering.push_back(i);
    }
    double t1 = (double)(-1.0 / (double)k);
    double sample_prob = pow(N, t1);
    // size_limit = 2 * math.pow(G.number_of_nodes(), 1 + 1 / k)
    double size_limit = 2.0 * pow(N, 1 + 1.0 / (double)k);
    ll index = 0;
    while (index < k - 1)
    {
        set<ll> sampled_centers;

        for (auto u : clustering)
        {
            if (rand() < sample_prob)
            {
                sampled_centers.insert(u);
            }
        }

        set<pair<ll, ll>> edges_to_add, edges_to_remove;
        vector<ll> new_clustering;

        for (int v = 1; v <= N; v++)
        {
            if (sampled_centers.find(clustering[v]) != sampled_centers.end())
                continue;

            pair<map<ll, ll>, map<ll, ll>> u = lightest_edge_dicts(v);
            map<ll, ll> lightest_edge_neighbour = u.first;
            map<ll, ll> lightest_edge_weight = u.second;

            set<ll> neighboring_sampled_centers;
            for (auto u : lightest_edge_weight)
            {
                if (sampled_centers.find(u.first) != sampled_centers.end())
                {
                    neighboring_sampled_centers.insert(u.first);
                }
            }

            if (neighboring_sampled_centers.size() == 0)
            {
                for (auto neighbour : lightest_edge_neighbour)
                {
                    edges_to_add.insert({v, neighbour.second});
                }

                for (auto neighbour : residual_graph[v])
                {
                    edges_to_remove.insert({v, neighbour.second});
                }
            }
            else
            {
                ll closest_center, closest_center_1 = *neighboring_sampled_centers.begin();
                for (auto u : lightest_edge_weight)
                {
                    if (u.second == closest_center_1)
                    {
                        closest_center = u.first;
                    }
                }

                ll closest_center_weight = lightest_edge_weight[closest_center];
                ll closest_center_neighbor = lightest_edge_neighbour[closest_center];

                edges_to_add.insert({v, closest_center_neighbor});
                new_clustering.push_back(closest_center);

                for (auto u : lightest_edge_weight)
                {
                    if (u.second < closest_center_weight)
                    {
                        edges_to_add.insert({v, lightest_edge_neighbour[u.first]});
                    }
                }

                for (auto u : residual_graph[v])
                {
                    ll neighbour_cluster = clustering[u.first];
                    ll neighbour_weight = lightest_edge_weight[neighbour_cluster];
                    if ((neighbour_cluster == closest_center) || (neighbour_weight < closest_center_weight))
                    {
                        edges_to_remove.insert({v, u.first});
                    }
                }
            }
        }
        if (edges_to_add.size() > size_limit)
            continue;

        for (auto u : edges_to_add)
        {
            add_edge_to_spanner(u.first, u.second, weight);
        }

        for (auto u : edges_to_remove)
        {
            ll start = u.first;
            ll end = u.second;

            for (auto v : residual_graph[start])
            {
                if (v.first == end)
                {
                    residual_graph[start].erase(find(residual_graph[start].begin(), residual_graph[start].end(), v));
                }
            }

            for (auto v : residual_graph[end])
            {
                if (v.first == start)
                {
                    residual_graph[end].erase(find(residual_graph[end].begin(), residual_graph[end].end(), v));
                }
            }
        }

        for (int j = 1; j <= N; j++)
        {
            for (auto u : clustering)
            {
                if (sampled_centers.find(u) != sampled_centers.end())
                {
                    new_clustering[j] = u;
                }
            }
        }

        for (int j = 1; j <= N; j++)
        {
            clustering.clear();
            for (auto u : new_clustering)
            {
                clustering.push_back(u);
            }
        }

        for (int j = 1; j <= N; j++)
        {
            for (auto u : residual_graph[j])
            {
                if (clustering[u.first] == clustering[j])
                {
                    ll start = u.first;
                    ll end = j;

                    for (auto v : residual_graph[start])
                    {
                        if (v.first == end)
                        {
                            residual_graph[start].erase(find(residual_graph[start].begin(), residual_graph[start].end(), v));
                        }
                    }

                    for (auto v : residual_graph[end])
                    {
                        if (v.first == start)
                        {
                            residual_graph[end].erase(find(residual_graph[end].begin(), residual_graph[end].end(), v));
                        }
                    }
                }
            }
        }

        for (int j = 1; j <= N; j++)
        {
            bool flag = true;

            if (find(clustering.begin(), clustering.end(), j) != clustering.end())
            {
                flag = false;
            }

            if (flag)
            {
                for (int j2 = 1; j2 <= N; j2++)
                {

                    for (auto v : residual_graph[j2])
                    {
                        if (v.first == j)
                        {
                            residual_graph[j2].erase(find(residual_graph[j2].begin(), residual_graph[j2].end(), v));
                        }
                    }
                }
                residual_graph[j].clear();
            }
        }

        for (int j = 1; j <= N; j++)
        {
            pair<map<ll, ll>, map<ll, ll>> m1 = lightest_edge_dicts(j);
            map<ll, ll> lightest_edge_neighbour = m1.second;
            for (auto u : lightest_edge_neighbour)
            {
                add_edge_to_spanner(j, u.second, weight);
            }
        }
        index++;
    }
}

int main()
{
    time_t start, end;
    ll i, j;
    time(&start);
    srand(time(0));
    generate_graph();
    spanner(4);
    time(&end);
    double time_taken = double(end - start);
    cout << "\nTime taken by program is : " << fixed
         << time_taken << setprecision(15);
    cout << " sec " << endl;
    return (0);
}


In [ ]:
%%shell

g++ Baswana_Sen_Serial.cpp -o Baswana_Sen_Serial
./Baswana_Sen_Serial